In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
import numpy as np
import json
import gc
from datetime import datetime
from pandas.io.json import json_normalize
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Y_pred_train
train_data = pd.read_csv("/kaggle/input/ga-customer-revenue-prediction/train.csv",dtype={'fullVisitorId': 'str'},low_memory=False)

In [ ]:
train_data.head()

In [ ]:
df=train_data.copy()

In [ ]:
#Creating array of column names that can be dropped from dataset
drop_columns=['date','fullVisitorId','sessionId','socialEngagementType','visitId','visitStartTime']

In [ ]:
#Extracting day of the week from dataset and creating new column
df['dayofweek']=pd.to_datetime(df.date, format='%Y%m%d', errors='ignore').dt.weekday

In [ ]:
#creating method to convert JSON columns to columns
def json_flatten1 (data,json_columns):
    try:
        for column in json_columns:
            column_to_df= json_normalize([json.loads(x) for x in data[column]])
            data=column_to_df
        return data
    
    except Exception as e:
            raise Exception()

In [ ]:
#converting device JSON column to columns
data_dev=json_flatten1(df,['device'])

In [ ]:
#Dropping the columns which have unique values =1
for col in data_dev.columns:
    if data_dev[col].nunique()==1:
        data_dev=data_dev.drop(col,axis=1)

In [ ]:
data_dev.head()

In [ ]:
#checking for is null values
data_dev.isna().sum()

In [ ]:
#creating method for marking as "Others" whose value_counts less than 100
def create_others(df,clm,leng):
    s = df[clm].value_counts()
    df[clm] = np.where(df[clm].isin(s.index[s < leng]), 'Other', df[clm])

In [ ]:
#Convertign values as others for "borwser" and "operating system" whose value_counts less tha 100
#replacing isMobile boolen columns with 1 and o
create_others(data_dev,'browser',100)
create_others(data_dev,'operatingSystem',100)
data_dev['isMobile'].replace({True:1,False:0},inplace=True)
#device cat fine

In [ ]:
#converting GeoNetwork JSON column to columns
data_geo=json_flatten1(df,['geoNetwork'])

In [ ]:
#Dropping the columns which have unique values =1
for col in data_geo.columns:
    if data_geo[col].nunique()==1:
        data_geo=data_geo.drop(col,axis=1)

In [ ]:
#checking for null values
data_geo.isna().sum()

In [ ]:
data_geo.head()

In [ ]:
#Repalcing the "(not set) and (not available in demo dataset)" with 0
data_geo.region.replace({'(not set)':0},inplace=True)
data_geo.region.replace({'not available in demo dataset':0},inplace=True)

data_geo.metro.replace({'(not set)':0},inplace=True)
data_geo.metro.replace({'not available in demo dataset':0},inplace=True)

data_geo.city.replace({'(not set)':0},inplace=True)
data_geo.city.replace({'not available in demo dataset':0},inplace=True)

data_geo.networkDomain.replace({'(not set)':0},inplace=True)
data_geo.networkDomain.replace({'not available in demo dataset':0},inplace=True)

#creating others for value_counts less than 100
create_others(data_geo,'region',100)
create_others(data_geo,'metro',100)
create_others(data_geo,'city',100)
create_others(data_geo,'networkDomain',100)

In [ ]:
data_geo.head()

In [ ]:
#converting totals JSON column to columns
data_tot=json_flatten1(df,['totals'])

In [ ]:
data_tot.head()

In [ ]:
#checking for null values
data_tot.isna().sum()

In [ ]:
#filling null values witj 0 or 1
data_tot.newVisits.fillna(value=1,inplace=True)
data_tot.pageviews.fillna(value=1,inplace=True)
data_tot.bounces.fillna(value=0,inplace=True)
data_tot.transactionRevenue.fillna(value=0,inplace=True)


In [ ]:
#converting trafficSource JSON column to columns
data_trf=json_flatten1(df,['trafficSource'])

In [ ]:
data_trf.head()

In [ ]:
#Dropping the columns which have unique values =1
for col in data_trf.columns:
    if data_trf[col].nunique()==1:
        data_trf=data_trf.drop(col,axis=1)

In [ ]:
#Checking for null values
data_trf.isna().sum()

In [ ]:
#Repalcing the "(not set) and (not available in demo dataset)" with 0
#creating others for value_counts less than 100
data_trf.campaign.replace({'(not set)':0},inplace=True)
create_others(data_trf,'campaign',100)
create_others(data_trf,'source',100)
data_trf.medium.replace({'(not set)':0},inplace=True)
data_trf.medium.replace({'(none)':0},inplace=True)
data_trf.keyword.fillna(value=0,inplace=True)
data_trf.keyword.replace({'(not provided)':0},inplace=True)
data_trf.keyword[data_trf.keyword != 0]=1

data_trf.referralPath.fillna(value=0,inplace=True)
data_trf.referralPath[data_trf.referralPath != 0]=1

data_trf['adwordsClickInfo.page'].fillna(value=0,inplace=True)

data_trf['adwordsClickInfo.slot'].fillna(value=0,inplace=True)
data_trf['adwordsClickInfo.slot'][data_trf['adwordsClickInfo.slot'] != 0]=1

data_trf['adwordsClickInfo.gclId'].fillna(value=0,inplace=True)
data_trf['adwordsClickInfo.gclId'][data_trf['adwordsClickInfo.gclId'] != 0]=1

data_trf['adwordsClickInfo.adNetworkType'].fillna(value=0,inplace=True)
data_trf['adwordsClickInfo.adNetworkType'][data_trf['adwordsClickInfo.adNetworkType'] != 0]=1

data_trf['adContent'].fillna(value=0,inplace=True)
data_trf['adContent'][data_trf['adContent'] != 0]=1


#create_others(data_trf,'referralPath',200)

In [ ]:
#merging all the JSON columns to dataset
df= df.merge(data_dev, right_index=True, left_index=True)
df= df.merge(data_geo, right_index=True, left_index=True)
df= df.merge(data_tot, right_index=True, left_index=True)
df= df.merge(data_trf, right_index=True, left_index=True)

In [ ]:
df.head()

In [ ]:
#dropping origin JSON columns
df=df.drop(columns=['device','geoNetwork','totals','trafficSource'])

In [ ]:
#Dropping unwanted columns
df=df.drop(columns=drop_columns)

In [ ]:
df1=df.copy()

In [ ]:
df1.isna().sum()

In [ ]:
df1.info()

In [ ]:
df2=df1.copy()

In [ ]:
#df2=df2.drop(columns=['adwordsClickInfo.page','adwordsClickInfo.slot','adwordsClickInfo.gclId','adwordsClickInfo.adNetworkType','adContent'])

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
#Checking for Cat values for label encoding
categorical_features = [
    _f for _f in df2.columns
    if(df1[_f].dtype == 'object')
]

In [ ]:
#label encoder initialising
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()

In [ ]:
for col in categorical_features:
    lbl.fit(list(df2[col].values.astype('str')) + list(df2[col].values.astype('str')))
    df2[col] = lbl.transform(list(df2[col].values.astype('str')))
    

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
#random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
Y=np.log1p(df2.transactionRevenue)
X=df2.drop(columns=['transactionRevenue'])

In [ ]:
#train_test_split
RF=RandomForestRegressor()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
RF.fit(X,Y)


In [ ]:
#predicting for test and train
Y_pred=RF.predict(X_test)
Y_pred_train=RF.predict(X_train)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

In [ ]:
print('test set'  ,r2_score(Y_test,Y_pred))
print('train set '  ,r2_score(Y_train,Y_pred_train))

In [ ]:
print('test set'  ,mean_squared_error(Y_test,Y_pred))
print('train set '  ,mean_squared_error(Y_train,Y_pred_train))

In [ ]:
print('test set' , np.sqrt(mean_squared_error(Y_test,Y_pred)))
print('train set '  ,np.sqrt(mean_squared_error(Y_train,Y_pred_train)))

In [ ]:
import pickle
file=open('C:\Lingesh_Vijay\Learning\ineuron\Internship\GoogleAnalytics.pkl','wb')
pickle.dump(RF,file)

In [ ]:
print(file.name)